# Enhancing German-to-English Machine Translation using Pre-trained Encoder and Decoder

## Motivation 

Our goal is to improve the accuracy of machine translation from German to English. To achieve this, we propose using a combination of pre-trained encoder and decoder models. BERT, a pre-trained encoder, excels at natural language understanding, while GPT, a pre-trained decoder, is proficient in natural language generation. We believe that combining these two models will yield results comparable to models like T5. To validate our approach, we will benchmark our translation performance against other state-of-the-art models on paperswithcode leaderboard.

We also explore the cross attention focusing on the final hidden state corresponding to BERT's CLS token because it is the sentence representation used to do next sentence prediction (NSP) during BERT's pre-training[1]. 

## Approach

We plan to utilize encoder-decoder transformers for our machine translation task. Specifically, we will employ the Huggingface's Encoder-Decoder Models library and initialize the model with a pre-trained encoder (German distilled BERT base) and a pre-trained decoder(English distilled GPT2). The library will initialize cross attention between the encoder and the decoder with random weights. Subsequently, fine-tuning will be performed on the model using the German-to-English dataset.

Because we are doing sequence-to-sequence task, we use the cross-entropy loss as the objective function:

Input sequence: X=(x1 ,x2 ,…,xT ) where T is the length of the input sequence. 

Target sequence: Y=(y1 ,y2 ,…,yU ) where U is the length of the target sequence. 

Predicted probabilities: P=(y^ 1 ,y^ 2 ,…,y^ U ) where y^ i  is the predicted probability distribution over the vocabulary for the i-th token in the target sequence. 

The cross-entropy loss is calculated as: 

CrossEntropy(Y,P)=−(1/U) ∑from i=1 in U ∑from j=1 in V  (yi,j x log(y^ i,j)) 


We implement cross-attention
mechanism focusing on the final hidden state corresponding to BERT’s CLS token by editing the
cross attention mask. Specifically, we set first element of the mask one and all else zero. Then the model can focus on the first hidden states and ignore the others.   

## Data 

The dataset used for this project was IWSLT 2014 German-to-English dataset (https://huggingface.co/datasets/bbaaaa/iwslt14-de-en). Hugging Face library was used to load the dataset. 


## Code 

For this project, we used homework 2 prefixtune as a template. To be precise, we used prefixtune.py, bleu.py, check.py, zipout.py, zipsrc.py, and requirement.txt.

## Experimental Setup 

We are evaluating the performance of our machine translation model on the task of translating German sentences into English. The metric used to guage the performance of our model is the Bilingual Evaluation Understudy Score (BLEU). We compared our full cross attention model with the one whose attention only to CLS token on the evaluation dataset. We use the same dataset to evaluate the choices of different hyper-parameters such as batch_size, epoch and predict() function parameters. 
In addition, the performance of our model was also measured using its perplexity on the evaluation dataset.

Our baseline is the model whose parameters are initialized from BERT and GPT2 and have not been fine-tuned.

We compared our BLEU score against methods such as Transformers, BiBERT and various other methods that were on the leaderboard of IWSLT2014 German-English on the test dataset (https://paperswithcode.com/sota/machine-translation-on-iwslt2014-german). 


## Results 


| different models performance on dev | BLEU score | PPL score |
|----------|----------|----------|
| baseline  | 1.9  | 8735  |
| trained for 1 epoch  | 26.46  | 1.35  |
| trained for 8 epoch  | 32.31  | 1.31  |
| trained for 1 epoch & attention to CLS only  | 1.6  | 1.85  |


| the model trained for 1 epoch, adjust predict() parameter | BLEU score |
|----------|----------|
| default: do_sample=True,<br> temperature=1.0,<br> max_new_tokens=50,<br> num_beams=5,<br> top_p=0.9  | 21.7  |
| do_sample = False (others unchanged)  | 21.8   |
| do_sample=True,<br> temperature=0.9,<br> no_repeat_ngram_size=2 (others unchanged)  | 23.0  |

| the model trained for 3 epoch, adjust predict() parameter | BLEU score |
|----------|----------|
| default: do_sample=True,<br> temperature=1.0,<br> max_new_tokens=50,<br> num_beams=5,<br> top_p=0.9  | 30.8  |
| do_sample = False (others unchanged)  | 30.8   |
| do_sample=True,<br> no_repeat_ngram_size=2 (others unchanged)  | 30.8   |

| paperswithcode benchmark models | BLEU score |
|----------|----------|
|BiBERT[7]| 38.61  |
|DynamicConv[6]| 35.2   |
|Ours| 34.94   |
|Attention Is All You Need[5]| 34.44   |
|Variational Attention[4]| 33.1   |
|Actor-Critic [Bahdanau2017][3]| 28.53   |

## Analysis of the Results

different models performance on dev

Our team obtained a baseline dev score of 1.9 for BLEU, and we were able to enhance that score to 32.31 after hyperparameter tuning and tweaking the predict function. For hyperparameter tuning, our group explored with different epochs, and batchsizes. We learned epoch 8 and batchsize 8 works best with our encoder decoder model. 
Furthermore, we tested with various values for num_beams, top_p, and temperature in the prediction. The best values for our model were num_beams = 5, top_p = 0.9, and temperature = 1.0. 

Cross-Attention on CLS token representation Experiment

The model with attention only CLS performs poorly because probably it’s not able to leverage the information from the input sequence effectively. Attending only to the CLS token won’t provide the necessary context and
alignment information between the source and target languages.
Moreover, the CLS token in BERT is a special token used for classification tasks and does not have any inherent representation of the input sequence’s content. During pre-training, the CLS token is used
for the next sentence prediction (NSP) task, which encourages the model to understand relationships
between sentence pairs but not specifically to encode sentence-level information[1].

adjustment of predict() parameter

When we first fine tune the models for just 1 epoch, adjusting predict() has some impact on the dev score. This is probably because the model might not have fully adapted to the task or the dataset. After we fine tuned the models for three epoch, adjusting predict() have negligible impact on the score. This can suggest that the model's internal representations have already become more refined and aligned with the task requirements.                

benchmark with paperwithcode models

Our model was able to achieve a final BLEU score of 34.94. It performed better than the base transformer model which achieved a score of 34.44 (Attention is all you need, 2017). In addition, our model also performed better over Actor-Critic (28.53), Variational Attention (33.1). BiBERT was the best performing model (38.61) and has a 3.67 BLEU score lead over our method. various other models such as DynamicConv (35.2) was also able to best our model. Overall, our method did not rank too high in terms of BLEU score.

## Future work 

As the first encoder decoder paper suggested, GPT2 as the pretrained decoder is not the best choice[2]. However, we believe an encoder decoder architecture is still worth exploring. We hope to explore other combinations of encoder decoder such as BERT encoder BERT decoder, and ALBERT encoder CTRL decoder in the future.

The pretrained models we used in this project are both trained on one language (German and English). However, researches like the one of mT5 suggested that multilingual models can outperform the monolingual models on the low resource language. We can explore the combination of multilingual models and fine tuning with multi language dataset. By doing this, we may achieve better results on low resource language. 

## Reference
[1] Jacob Devlin, Ming-Wei Chang, Kenton Lee, and Kristina N. Toutanova. Bert: Pre-training of
deep bidirectional transformers for language understanding. 2018.

[2] Sascha Rothe, Shashi Narayan, and Aliaksei Severyn. Leveraging pre-trained checkpoints for
sequence generation tasks. Transactions of the Association for Computational Linguistics, 8:264–
280, 2020. https://doi.org/10.1162/tacla00313

[3] Dzmitry Bahdanau, Philemon Brakel, Kelvin Xu, Anirudh Goyal, Ryan Lowe, Joelle Pineau,
Aaron Courville, and Yoshua Bengio. An actor-critic algorithm for sequence prediction. arXiv
preprint arXiv:1607.07086, 2016.

[4] Yuntian Deng, Yoon Kim, Justin Chiu, Demi Guo, and Alexander Rush. Latent alignment and
variational attention. Advances in neural information processing systems, 31, 2018.

[5] Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N Gomez, Łukasz
Kaiser, and Illia Polosukhin. Attention is all you need. Advances in neural information processing
systems, 30, 2017.

[6] Felix Wu, Angela Fan, Alexei Baevski, Yann N Dauphin, and Michael Auli. Pay less attention with
lightweight and dynamic convolutions. arXiv preprint arXiv:1901.10430, 2019.

[7] Haoran Xu, Benjamin Van Durme, and Kenton Murray. Bert, mbert, or bibert? a study on contextu-
alized embeddings for neural machine translation. arXiv preprint arXiv:2109.04588, 2021.

## Group work

* kerryz implemented baseline, fine tune the model, write up
* rxa13 writeup, attempted baseline, perplexity, research 
* lwa write up, poster